<b>Import data<b>

In [1]:
import pandas

articles1 = pandas.read_csv("/Users/yuji/Desktop/COMP562/FinalProject/articles1.csv")
# articles2 = pandas.read_csv("/Users/yuji/Desktop/COMP562/FinalProject/articles2.csv")
# articles3 = pandas.read_csv("/Users/yuji/Desktop/COMP562/FinalProject/articles3.csv")
fakearticle = pandas.read_csv("/Users/yuji/Desktop/COMP562/FinalProject/fake.csv")

<b>Process article 1<b>

In [2]:
import numpy as np

In [3]:
articles1_content = articles1.drop(["Unnamed: 0", "id", "publication","author","date","year","month","url"], axis = 1)
articles1_array = np.asarray(articles1_content)

# articles3_content = articles3.drop(["Unnamed: 0", "id", "publication","author","date","year","month","url"], axis = 1)
# articles3_array = np.asarray(articles3_content)
fakearticle_content = fakearticle.drop(['uuid', 'ord_in_thread','published','author','language','crawled','site_url','country','domain_rank','thread_title','spam_score','main_img_url','replies_count','participants_count','likes','comments','shares','type'], axis = 1)
fake_array = np.asarray(fakearticle_content)[:, :]

<b>randomly choose 1000 data<b>

In [4]:
real_size = np.shape(articles1_array)[0]
real_index = np.random.choice(real_size, 100)
real_data = articles1_array[real_index, :]
print(np.shape(real_data))

fake_size = np.shape(fake_array)[0]
fake_index = np.random.choice(fake_size, 100)
fake_data = fake_array[fake_index, :]
print(np.shape(fake_data))

data = np.concatenate((real_data, fake_data),axis = 0)

(100, 2)
(100, 2)


In [18]:
test_size = np.shape(articles2_array)[0]
test_index = np.random.choice(real_size, 1600)
test_data = articles2_array[test_index, :]

fake_index2 = np.random.choice(fake_size, 400)
test_fake = fake_array[fake_index, :]

test = np.concatenate((test_data, test_fake), axis = 0)

In [19]:
print(np.shape(data))

(2000, 2)


<b> create dictionary <b>

In [5]:
dict = {}
count = 0
list = []

In [6]:
import re
# datas = [articles1_array, articles2_array, articles3_array, fake_array]
# datas = [articles1_array, fake_array]
N = np.shape(data)[0];
for i in range(N):
    if((type(data[i,0]) is not str)):
        continue;
    if((type(data[i,1]) is not str)):
        continue;
    title = re.sub(r'[^\w\s]','',data[i,0]).upper().split()
    content = re.sub(r'[^\w\s]','',data[i,1]).upper().split()
    content.extend(title)
    for word in content:
        if(dict.get(word) is None):
            dict[word]=count;
            count+=1

In [22]:
print(len([*dict]))

48983


<b>Create Feature<b>

In [23]:
count = 0
F = len([*dict])
N = np.shape(data)[0];
X = np.zeros((N, F))

for i in range(N):
    if((type(data[i,0]) is not str)):
        continue;
    if((type(data[i,1]) is not str)):
        continue;
    title = re.sub(r'[^\w\s]','',data[i,0]).upper().split()
    content = re.sub(r'[^\w\s]','',data[i,1]).upper().split()
    content.extend(title)
    for word in content:
        X[count, dict[word]] += 1
    count+=1

<b>create test feature<b>

In [25]:
count = 0
F = len([*dict])
N = np.shape(test)[0];
X_test = np.zeros((N, F))

for i in range(N):
    if((type(test[i,0]) is not str)):
        continue;
    if((type(test[i,1]) is not str)):
        continue;
    title = re.sub(r'[^\w\s]','',test[i,0]).upper().split()
    content = re.sub(r'[^\w\s]','',test[i,1]).upper().split()
    content.extend(title)
    for word in content:
        if(dict.get(word) is not None):
            X_test[count, dict[word]] += 1
    count+=1

<b> create label<b>

In [27]:
lab = [1]*1600
neg = [0]*400
lab.extend(neg)
lab = np.asarray(lab)
print(np.shape([lab.T]))
X = np.concatenate((X, np.transpose([lab.T])), axis = 1)
X_test = np.concatenate((X_test, np.transpose([lab.T])), axis = 1)

(1, 2000)


In [28]:
np.random.shuffle(X)
np.random.shuffle(X_test)
np.save("/Users/yuji/Desktop/train.npy", X[:,:np.shape(X)[1]-2])
print("done")
np.save("/Users/yuji/Desktop/train_label.npy", X[:,np.shape(X)[1]-1])
print("done")

done
done


In [29]:
np.save("/Users/yuji/Desktop/test.npy", X_test[:,:np.shape(X_test)[1]-2])
np.save("/Users/yuji/Desktop/test_label.npy", X_test[:,np.shape(X_test)[1]-1])

[24398 15038  1858 10329 32285 49949 20399 49157 16408 21448]
[5272 1605 7758 1062 6856 8784 3427 5548 2148 1466]
[26538 37603 46688 26642 29729 31483  8312 37648 46193 45360]
[8422 5993 8576 5935 9664 7215 4017 5195 4741 4376]
